In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import exposure, measure
from scans import *
from tumor_centers import *
import os

In [2]:
def plot_slice(slice, title):
    plt.imshow(slice, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()
    
def load_data(scan_path, label_path):
    scan = nib.load(scan_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    return scan, label

def select_slice(scan, label, slice_idx):
    return scan[:, :, slice_idx], label[:, :, slice_idx]

def preprocess_slice(slice):
    p2, p98 = np.percentile(slice, (2, 98))
    return exposure.rescale_intensity(slice, in_range=(p2, p98))

def region_growing(image, seed):
    seg_result = np.zeros_like(image)
    seg_result[seed] = 1
    points = [seed]
    seedval = image[seed]
    threshval = 0.1 * np.max(image)
    
    while points:
        x, y = points.pop(0)
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < image.shape[0] and 0 <= ny < image.shape[1]:
                    pointval = image[nx, ny]
                    if seg_result[nx, ny] == 0 and seedval - threshval <= pointval <= seedval + threshval:
                        seg_result[nx, ny] = 1
                        points.append((nx, ny))
    return seg_result

def dice_coefficient(segmented, ground_truth):
    intersection = np.logical_and(segmented, ground_truth)
    return (2. * intersection.sum()) / (segmented.sum() + ground_truth.sum())

In [3]:
dataset_path = 'D:\Dosyalar\dataset\Task06_Lung'
images_train_dir = os.path.join(dataset_path, 'imagesTr')
labels_train_dir = os.path.join(dataset_path, 'labelsTr')

In [4]:
global_dscs = []
global_sensitivities = []
global_specificities = []
global_accuracies = []
global_weights = []
global_TP = 0
global_TN = 0
global_FP = 0
global_FN = 0

In [9]:
with open('results.txt', 'w') as results_file:
    for scan_number in INDEXES[-21:]:
        scan_path = os.path.join(images_train_dir, f'lung_{scan_number}.nii.gz')
        label_path = os.path.join(labels_train_dir, f'lung_{scan_number}.nii.gz')
        scan, label = load_data(scan_path, label_path)
        
        slice_indexes = globals()[f'scan_{scan_number}']
        tumor_centers = globals()[f'tumor_centers_{scan_number}']

        for i, slice_idx in enumerate(slice_indexes):
            slice, slice_label = select_slice(scan, label, slice_idx)

            slice_preprocessed = preprocess_slice(slice)

            center = tumor_centers[i]
            seed_point = (int(center[1]), int(center[0])) # Swap x, y
            segmented_mask = region_growing(slice_preprocessed, seed_point)

            TP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 1))
            TN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 0))
            FP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 0))
            FN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 1))
            
            global_TP += TP
            global_TN += TN
            global_FP += FP
            global_FN += FN

            dsc = dice_coefficient(segmented_mask, slice_label)
            print(f'Scan {scan_number} - Slice {slice_idx}: {dsc}')
            sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
            specificity = TN / (TN + FP) if TN + FP > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN > 0 else 0

            global_dscs.append(dsc)
            global_sensitivities.append(sensitivity)
            global_specificities.append(specificity)
            global_accuracies.append(accuracy)
            global_weights.append(np.sum(segmented_mask))

Scan 062 - Slice 145: 0.0380952380952381
Scan 062 - Slice 146: 0.01932367149758454
Scan 062 - Slice 147: 0.006309148264984227
Scan 062 - Slice 148: 0.2735674676524954
Scan 062 - Slice 149: 0.6259946949602122
Scan 062 - Slice 150: 0.6868421052631579
Scan 062 - Slice 151: 0.5991189427312775
Scan 062 - Slice 152: 0.12479474548440066
Scan 062 - Slice 153: 0.6201780415430267
Scan 062 - Slice 154: 0.7052631578947368
Scan 062 - Slice 155: 0.5642458100558659
Scan 062 - Slice 156: 0.007207207207207207
Scan 062 - Slice 157: 0.39920948616600793
Scan 062 - Slice 158: 0.4523809523809524
Scan 062 - Slice 159: 0.14285714285714285
Scan 062 - Slice 160: 0.5
Scan 062 - Slice 161: 0.012320328542094456
Scan 064 - Slice 119: 0.48148148148148145
Scan 064 - Slice 120: 0.06666666666666667
Scan 064 - Slice 121: 0.23937007874015748
Scan 064 - Slice 122: 0.3434146341463415
Scan 064 - Slice 123: 0.4624277456647399
Scan 064 - Slice 124: 0.01486988847583643
Scan 064 - Slice 125: 0.0040844111640571815
Scan 064 - Sli

In [7]:
overall_weighted_mean_dsc = np.average(global_dscs, weights=global_weights)
overall_mean_dsc = np.mean(global_dscs) 
overall_weighted_std_dsc = np.sqrt(np.average((global_dscs - overall_weighted_mean_dsc) ** 2, weights=global_weights))
overall_weighted_mean_sensitivity = np.average(global_sensitivities, weights=global_weights)
overall_weighted_std_sensitivity = np.sqrt(np.average((global_sensitivities - overall_weighted_mean_sensitivity) ** 2, weights=global_weights))
overall_weighted_mean_specificity = np.average(global_specificities, weights=global_weights)
overall_weighted_std_specificity = np.sqrt(np.average((global_specificities - overall_weighted_mean_specificity) ** 2, weights=global_weights))
overall_weighted_mean_accuracy = np.average(global_accuracies, weights=global_weights)
overall_weighted_std_accuracy = np.sqrt(np.average((global_accuracies - overall_weighted_mean_accuracy) ** 2, weights=global_weights))


In [10]:
overall_mean_dsc = np.mean(global_dscs)
overall_mean_sensitivity = np.mean(global_sensitivities)
overall_mean_specificity = np.mean(global_specificities)
overall_mean_accuracy = np.mean(global_accuracies)
print("Overall Mean DSC:", overall_mean_dsc)
print("Overall Mean Sensitivity:", overall_mean_sensitivity)
print("Overall Mean Specificity:", overall_mean_specificity)
print("Overall Mean Accuracy:", overall_mean_accuracy)


Overall Mean DSC: 0.19502738264871886
Overall Mean Sensitivity: 0.17891314116892745
Overall Mean Specificity: 0.9953573494670304
Overall Mean Accuracy: 0.9930248495973186


In [25]:
true_positive_rate = global_TP / (global_TP + global_FN) * 100
true_negative_rate = global_TN / (global_TN + global_FP) * 100
false_positive_rate = global_FP / (global_FP + global_TN) * 100
false_negative_rate = global_FN / (global_FN + global_TP) * 100

In [8]:
# Print or save overall results
print(f'Overall Weighted Mean DSC: {overall_weighted_mean_dsc}, Standard Deviation: {overall_weighted_std_dsc}')
print(f'Overall Mean DSC: {overall_mean_dsc}')
print(f'Overall Weighted Mean Sensitivity: {overall_weighted_mean_sensitivity}, Standard Deviation: {overall_weighted_std_sensitivity}')
print(f'Overall Weighted Mean Specificity: {overall_weighted_mean_specificity}, Standard Deviation: {overall_weighted_std_specificity}')
print(f'Overall Weighted Mean Accuracy: {overall_weighted_mean_accuracy}, Standard Deviation: {overall_weighted_std_accuracy}')

Overall Weighted Mean DSC: 0.015607936456094285, Standard Deviation: 0.059966219066652705
Overall Mean DSC: 0.13350863789934908
Overall Weighted Mean Sensitivity: 0.7211122198981109, Standard Deviation: 0.2521661774139204
Overall Weighted Mean Specificity: 0.9236333659042231, Standard Deviation: 0.025747239031455323
Overall Weighted Mean Accuracy: 0.9234296842060804, Standard Deviation: 0.02526283233968762


In [27]:
print(f'True Positive Rate: {true_positive_rate}%')
print(f'True Negative Rate: {true_negative_rate}%')
print(f'False Positive Rate: {false_positive_rate}%')
print(f'False Negative Rate: {false_negative_rate}%')

True Positive Rate: 10.124001817827697%
True Negative Rate: 99.27432554351965%
False Positive Rate: 0.7256744564803563%
False Negative Rate: 89.8759981821723%
